In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from PIL import Image

In [ ]:
from tensorflow.keras.layers import Input,Dense,Flatten,BatchNormalization,GlobalAveragePooling2D,Dropout,Conv2D,MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [ ]:
genobj=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,rescale=1/255.0,validation_split=0.2)

In [ ]:
np.array(Image.open('/kaggle/input/hand-gesture-recognition/Dataset_Binary/Dataset_Binary/eight/img 0.jpg')).shape

In [ ]:
BATCH_SIZE=20

In [ ]:
train_obj=genobj.flow_from_directory('/kaggle/input/hand-gesture-recognition/Dataset_Binary/Dataset_Binary',target_size=(300,300),batch_size=BATCH_SIZE,subset='training')

In [ ]:
val_obj=genobj.flow_from_directory('/kaggle/input/hand-gesture-recognition/Dataset_Binary/Dataset_Binary',target_size=(300,300),batch_size=BATCH_SIZE,subset='validation')

In [ ]:
train_obj.classes

In [ ]:
def plot_image():
    list_image_label=[]
    for i in range(9):
        batch=next(train_obj)
        rndm_no=np.random.randint(0,len(next(train_obj)[0]))
        list_image_label.append((batch[0][rndm_no],batch[1][rndm_no]))
    return list_image_label

In [ ]:
list_img=plot_image()

In [ ]:
fig,axes=plt.subplots(3,3)
axes=axes.flatten()
count=0
for i,j in list_img:
    
    plt.figure()
    label=np.where(j==1)[0]
    axes[count].imshow(i, cmap='gray')
    axes[count].set_title(f'Label: {label}')
    axes[count].axis('off')
    count+=1
plt.tight_layout()
train_obj.class_indices

In [ ]:
def draw_single(obj):
    batch=next(obj)
    image,label=batch[0][0],batch[1][0]
    return image,label

In [ ]:
img,lbl=draw_single(train_obj)

In [ ]:
plt.imshow(img)
plt.axis('off')

In [ ]:
input_shape=img.shape

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
steps_per_epoch=12000//BATCH_SIZE

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Add custom head for your specific task (e.g., classification)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)

# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
r=model.fit(train_obj,batch_size=BATCH_SIZE,steps_per_epoch=steps_per_epoch,epochs=10,validation_data=val_obj)

In [ ]:
from tensorflow.keras.models import save_model,load_model

In [ ]:
save_model(model,'/kaggle/working/modelHG_BIN.h5')

In [ ]:
train_loss = r.history['loss']
val_loss = r.history['val_loss']

# Create a plot
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
best_epoch_index = r.history['val_loss'].index(min(r.history['val_loss']))
best_epoch_index

In [ ]:
print("Train Loss:",r.history['loss'][0])
print("Train Accuracy:",r.history['accuracy'][0])
print("Validation Loss:",r.history['val_loss'][0])
print("Validation Accuracy:",r.history['val_accuracy'][0])

In [ ]:
# Retrieve the true labels
labels = val_obj.classes if hasattr(val_obj, 'classes') else None

# Make predictions on the validation set
y_pred = model.predict(val_obj)

# If labels attribute is not present in val_obj, an alternative is:
# Convert predicted probabilities to class predictions
if labels is None:
    print("Warning: 'val_obj' does not have a 'classes' attribute. Please verify the source of true labels.")

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import numpy as np

# Display the confusion matrix and classification report
ConfusionMatrixDisplay.from_predictions(labels, y_pred)
print(classification_report(labels, y_pred))

# UPGRADED CODE

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import keras
import matplotlib.pyplot as plt 
import os 
import cv2 
import numpy as np
import pandas as pd

# Import of keras model and hidden layers for our convolutional network
from keras.layers import Conv2D, Activation, MaxPool2D, Dense, Flatten, Dropout

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
CATEGORIES = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 
    'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
]

IMG_SIZE = 50

data_path = "/kaggle/input/indian-sign-language-islrtc-referred/original_images"

In [ ]:
# for binary images 
# Load image data
image_data = []
for category in CATEGORIES:
    class_index = CATEGORIES.index(category)
    path = os.path.join(data_path, category)  # Assuming categories are directly under data_path

    # Check if path exists to avoid FileNotFoundError
    if not os.path.isdir(path):
        print(f"Directory not found: {path}")
        continue

    for img in os.listdir(path):
        try:
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            if img_arr is not None:  # Check if the image was read successfully
                resized_img = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                image_data.append([resized_img, class_index])
        except Exception as e:
            print(f"Error reading image {img}: {e}")
            pass

# Display first item in image_data
print(image_data[0])

In [ ]:
image_data = []
for dr in os.listdir(data_path):
    for category in CATEGORIES:
        class_index = CATEGORIES.index(category)
        path = os.path.join(data_path, dr, category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                image_data.append([cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE)), class_index])
            except Exception as e:
                pass
image_data[0]

In [ ]:
# Initialize list to store image data and labels
image_data = []

# Iterate over each category directly in `original_images`
for category in CATEGORIES:
    class_index = CATEGORIES.index(category)
    path = os.path.join(data_path, category)  # Construct the path for each category
    
    # Check if the directory exists
    if not os.path.exists(path):
        print(f"Directory not found: {path}")
        continue  # Skip this category if the folder is missing

    # Load images from the category directory
    for img in os.listdir(path):
        try:
            img_path = os.path.join(path, img)
            img_arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Ensure the image was loaded successfully
            if img_arr is not None:
                resized_img = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                image_data.append([resized_img, class_index])
            else:
                print(f"Failed to load image: {img_path}")
        except Exception as e:
            print(f"Error processing image {img} in {path}: {e}")

# Display the first item in the image_data list to verify
if image_data:
    print(image_data[0])
else:
    print("No images loaded.")

In [ ]:
import random
random.shuffle(image_data)

In [ ]:
input_data = []
label = []
for X, y in image_data:
    input_data.append(X)
    label.append(y)

In [ ]:
label[:10]

In [ ]:
plt.figure(1, figsize=(10,10))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.imshow(image_data[i][0], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(CATEGORIES[label[i]][3:])
plt.show()

In [ ]:
input_data = np.array(input_data)
label = np.array(label)
input_data = input_data/255.0
input_data.shape

In [ ]:
import keras
from keras.utils import to_categorical
label = to_categorical(label)

In [ ]:
input_data.shape = (-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_data, label,test_size=0.2, random_state=42, shuffle=True)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Define the model
model = keras.models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Activation('relu'))

model.add(Conv2D(filters=32, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(36, activation='softmax'))  # Update to 15 units to match number of classes

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=6, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Save the model
model.save("hand_gesture_recognition_model_b.h5")

In [ ]:
model.save_weights("hand_gesture_recognition_weights.weights.h5")

In [ ]:
# Plot model accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
train_acc = history.history['accuracy']
train_loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

index_loss = np.argmin(val_loss)
index_acc = np.argmax(val_acc)

val_lowest = val_loss[index_loss]
val_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(train_acc))]

loss_label = f'Best Epoch = {str(index_loss + 1)}'
acc_label = f'Best Epoch = {str(index_acc + 1)}'

plt.figure(figsize= (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs , train_loss , 'r', label = 'Training Loss')
plt.plot(Epochs , val_loss , 'g' , label = 'Validation Loss')
plt.scatter(index_loss +1 , val_lowest , s = 150 , c = 'blue' , label = loss_label)
plt.title('Training vs Validation (loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs , train_acc , 'r', label= 'Training Accuracy')
plt.plot(Epochs , val_acc , 'g' , label = 'Validation Accuracy')
plt.scatter(index_acc + 1 , val_highest , s= 150 , c = 'blue' , label= acc_label)
plt.title('Training vs Validation (Accuracy)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show();

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test accuracy: {:2.2f}%'.format(test_accuracy*100))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Get model predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true = np.argmax(y_test, axis=1)          # Convert one-hot encoded labels to class labels

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Generate model predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true = np.argmax(y_test, axis=1)          # Convert one-hot encoded labels to class labels

# Generate and display confusion matrix with enhanced visibility
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(30, 20))  # Increase figure size for better clarity
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CATEGORIES)
disp.plot(cmap='Blues', xticks_rotation=45)  # Rotate x-axis labels for better readability

# Adjust text size and color bar for clarity
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Access the AxesImage object directly from the display
plt.colorbar(disp.im_, fraction=0.046, pad=0.04).set_label('Count', size=12)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true = np.argmax(y_test, axis=1)          # Convert one-hot encoded labels to class labels

# Generate the classification report
report = classification_report(y_true, y_pred_classes, target_names=CATEGORIES)
print(report)

In [ ]:
"""import cv2
import numpy as np
from keras.models import load_model

# Load the saved model
model = load_model("hand_gesture_recognition_model.h5")

# Define constants
IMG_SIZE = 50
CATEGORIES = ['down', 'eight', 'five', 'four', 'left', 'nine', 'one', 'right', 'seven', 'six', 'stop', 'three', 'two', 'up', 'zero']

def prepare_image(filepath):
    """
    Preprocesses the image at 'filepath' to the format required by the model.
    """
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    if img_array is None:
        print(f"Error: Unable to read image at {filepath}")
        return None

    resized_img = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # Resize to match model input size
    normalized_img = resized_img / 255.0  # Normalize pixel values to [0,1]
    reshaped_img = np.reshape(normalized_img, (1, IMG_SIZE, IMG_SIZE, 1))  # Reshape for model input
    return reshaped_img

def predict_gesture(filepath):
    """
    Loads an image, preprocesses it, and predicts the gesture category.
    """
    # Prepare the image
    prepared_img = prepare_image(filepath)
    if prepared_img is None:
        return "Invalid image."

    # Predict the class
    predictions = model.predict(prepared_img)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    predicted_label = CATEGORIES[predicted_class]

    # Print the result
    print(f"Predicted Gesture: {predicted_label} | Confidence: {confidence:.2f}")
    return predicted_label, confidence

# Test the function with a new image path
image_path = "/kaggle/input/indian-sign-language-islrtc-referred/original_images/2/1000.jpg"  # Update with your image path
predicted_label, confidence = predict_gesture(image_path)

# Save the image with the prediction text overlay
if predicted_label:
    img = cv2.imread(image_path)
    if img is not None:
        output_image_path = "predicted_gesture.jpg"
        cv2.putText(img, f"{predicted_label} ({confidence:.2f})", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imwrite(output_image_path, img)
        print(f"Image with prediction saved at {output_image_path}")
    else:
        print("Failed to load the image for overlay.")
        """